### Anime Recommendation System

In [1]:
#Import the necessary libraries: 

import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

import nltk
import nltk.corpus
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saiya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### 1.) Data Observations

Data retrieved from: https://www.kaggle.com/datasets/dbdmobile/myanimelist-dataset/data . The data is from myAnimeList

First, we will observe the characteristics of this data and jot down our observations by converting the following CSV file into a Pandas DataFrame. 


In [2]:
animedf = pd.read_csv('anime-dataset-2023.csv')
animedf

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,无脑魔女,UNKNOWN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,ONA,15.0,"Jul 4, 2023 to ?",...,UNKNOWN,Web manga,Unknown,PG-13 - Teens 13 or older,UNKNOWN,24723,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,UNKNOWN,"Action, Adventure, Fantasy",No description available for this anime.,ONA,18.0,"Jul 27, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,The First Order,第一序列,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,ONA,16.0,"Jul 19, 2023 to ?",...,UNKNOWN,Web novel,Unknown,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,UNKNOWN,UNKNOWN,A music video for the song Bokura no Saishuu S...,Music,1.0,"Apr 23, 2022",...,UNKNOWN,Original,3 min,PG-13 - Teens 13 or older,0.0,0,0,UNKNOWN,0,https://cdn.myanimelist.net/images/anime/1931/...


In [3]:
animedf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   anime_id      24905 non-null  int64 
 1   Name          24905 non-null  object
 2   English name  24905 non-null  object
 3   Other name    24905 non-null  object
 4   Score         24905 non-null  object
 5   Genres        24905 non-null  object
 6   Synopsis      24905 non-null  object
 7   Type          24905 non-null  object
 8   Episodes      24905 non-null  object
 9   Aired         24905 non-null  object
 10  Premiered     24905 non-null  object
 11  Status        24905 non-null  object
 12  Producers     24905 non-null  object
 13  Licensors     24905 non-null  object
 14  Studios       24905 non-null  object
 15  Source        24905 non-null  object
 16  Duration      24905 non-null  object
 17  Rating        24905 non-null  object
 18  Rank          24905 non-null  object
 19  Popu

In [4]:
#Lets understand what kind of columns we have. We only need certain kinds of columns to gain important info for our rec system
animedf.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Type', 'Episodes', 'Aired', 'Premiered', 'Status',
       'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating',
       'Rank', 'Popularity', 'Favorites', 'Scored By', 'Members', 'Image URL'],
      dtype='object')

In [5]:
animedf['Type'] #type columns indicates if its a movie or tv show. 

0           TV
1        Movie
2           TV
3           TV
4           TV
         ...  
24900      ONA
24901      ONA
24902      ONA
24903    Music
24904    Music
Name: Type, Length: 24905, dtype: object

In [6]:
animedf['Type'].value_counts()

Type
TV         7597
Movie      4381
OVA        4076
ONA        3533
Music      2686
Special    2558
UNKNOWN      74
Name: count, dtype: int64

Here we are exploring the "Type" column more in-depth because we want our anime recommendation system to only focus on TV, Movie, OVA, ONA and Specials. We notice here that we have "UNKNOWNS" and "MUSIC" which in a content based recommendation system may not be as significant to the anime experience. We may need to perform some data cleaning and keep necessary information columns to help build our recommendation system.

In [7]:
#Check for null values:
animedf.isnull().sum()

anime_id        0
Name            0
English name    0
Other name      0
Score           0
Genres          0
Synopsis        0
Type            0
Episodes        0
Aired           0
Premiered       0
Status          0
Producers       0
Licensors       0
Studios         0
Source          0
Duration        0
Rating          0
Rank            0
Popularity      0
Favorites       0
Scored By       0
Members         0
Image URL       0
dtype: int64

In [8]:
animedf.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...


So far, we can indicate the name, score, scored by, genre type, synopsis (summary), type, rating, and favorites. Popularity and rank might be redundant here and might lead to multicollinearity. I believe these will be the most necessary columns in order to build the recommender system. For now we will also keep anime_id if we need to reference back to a unique case or outlier. But it will not be included in the model.

In [9]:
animedf[animedf['Name'] == 'One Piece'] #understanding the "Score" and "Scored By"

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
11,21,One Piece,One Piece,ONE PIECE,8.69,"Action, Adventure, Fantasy","Gol D. Roger was known as the ""Pirate King,"" t...",TV,UNKNOWN,"Oct 20, 1999 to ?",...,Toei Animation,Manga,24 min,PG-13 - Teens 13 or older,55.0,20,198986,1226493.0,2168904,https://cdn.myanimelist.net/images/anime/6/732...


##### Other observations on data before cleaning:

1.) Dataset focuses on the anime type and its other characteristics such as episode amount, release date, studio creation, etc. It provides in-depth information on the anime listed. 

2.) We may face limitations due to the dataset being focused on the 2023 lineup of data in MyAnimeList and may not be up to date as well as its a dataset from Kaggle. It may not reflect current anime trends today but understand anime trends last year among users on that website.

3.) All of the column values are objects. 

In [10]:
#Now lets clean up our data to keep relevant features. Do this by making new df called animedf_clean

#1. Drop the "English Name" and "Other Name" as well as other unnecessary columns. We notice that "English Name" has "UNKNOWNS" Listed. So we should keep "Name" only. 
animedf_clean = animedf.drop(['English name','Other name', 'Episodes', 'Studios','Source','Duration','Rank','Popularity','Members', 'Premiered', 'Licensors','Source', 'Producers', 'Aired', 'Status'], axis=1)
animedf_clean.columns

Index(['anime_id', 'Name', 'Score', 'Genres', 'Synopsis', 'Type', 'Rating',
       'Favorites', 'Scored By', 'Image URL'],
      dtype='object')

In [11]:
animedf_clean['Synopsis'].tail(50)

24855          Music video for the song TRIPPER by Kankan.
24856    Music video for the song Fins by Tsunomaki Wat...
24857             No description available for this anime.
24858    Music video for the song Twilight (#1 Abstract...
24859             No description available for this anime.
24860    Music video for the song Sayonara Wonderland b...
24861    In the majestic fantasy world, how does the pr...
24862    Music video for the song Denki Groove 30-shuun...
24863     Music video for the song Gin Kkum by SE SO NEON.
24864    Music video for the song Another day, another ...
24865    Music video for the original song Baka Bakka b...
24866             No description available for this anime.
24867      Second season of Boku no Kokoro no Yabai Yatsu.
24868    Seiichi is bored with his routine life as a te...
24869             No description available for this anime.
24870             No description available for this anime.
24871             No description available for this anim

In [12]:
#Filter and remove "UNKNOWN" and "Music" types we only want our recommender system to recommend users movies, tv shows, specials, etc.
animedf_clean = animedf_clean[(animedf_clean['Type'] != 'UNKNOWN') & (animedf_clean['Type'] != 'Music')]
animedf_clean['Type'].value_counts()

Type
TV         7597
Movie      4381
OVA        4076
ONA        3533
Special    2558
Name: count, dtype: int64

In [13]:
#Since our content-based recommendation system will depend on Genres and Synopsis, we should check for any "UNKNOWN" values.

# CASE 1: Unknown genres, however, synopsis is present. For a situation like this we can impute and using NLP techniques to assume the genre.
# Lowercase the synopsis and remove punctuation
missing_check1 = animedf_clean[(animedf_clean['Genres'] == 'UNKNOWN') & (animedf_clean['Synopsis'] != 'No description available for this anime.')]
missing_check1['Synopsis'] = missing_check1['Synopsis'].str.lower().str.replace(r'[^\w\s]', '', regex=True)

# Implement the stopwords:
stopwords = nltk.corpus.stopwords.words('english')
missing_check1['Synopsis'] = missing_check1['Synopsis'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word not in stopwords]))

#Lemmatization: 

lemmatizer = nltk.WordNetLemmatizer()

missing_check1['Synopsis'] = missing_check1['Synopsis'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in nltk.word_tokenize(x)]))


#Anime genres and their keywords to correspond to each of them. Source: ChatGPT. We can set one genre for an unknown genre based on synopsis keywords matching
#with anime keyword.



                

C:\Users\saiya\AppData\Local\Temp\ipykernel_3616\3533672805.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_check1['Synopsis'] = missing_check1['Synopsis'].str.lower().str.replace(r'[^\w\s]', '', regex=True)
C:\Users\saiya\AppData\Local\Temp\ipykernel_3616\3533672805.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_check1['Synopsis'] = missing_check1['Synopsis'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(x) if word not in stopwords]))
C:\Users\saiya\AppData\Local\T

In [14]:
# CASE 2: Known Genre, however, no synopsis. -> Remove them

# CASE 3: Unknown genre and unknown synopsis -> Remove them

In [15]:
#Next we perform feature generation/engineering to make a "Tags" column. We merge the "Synopsis" and "Genres" columns together.

tags = (animedf_clean['Genres'] + " " + animedf_clean['Synopsis'])
animedf_clean['Tags'] = tags